In [ ]:
import alg_gen
import importlib

importlib.reload(alg_gen)

from alg_gen import AlgGen
ag = AlgGen()

In [ ]:
import pandas as pd
from IPython.display import clear_output
import csv

In [ ]:
solutions = ag.generate_solutions_twizzle("B' U R B' R' B U L' B' L R B R' U B", max_depth=12, memory=8192)

In [ ]:
setups = pd.read_csv("L3T_setups.csv")
data = {}

for i in range(5):
    clear_output(wait=True)
    grouped_solutions, dicpos = [],[]
    min_length = 99
    grouped_ranked = []
    grouped_ranked_rotations = []
    solutions = ag.generate_solutions_twizzle(setups.iloc[i]["Setup"], max_depth=14, memory=8192)


    for solution in solutions:
        length = len(solution.split())
        if length in dicpos:
            grouped_solutions[dicpos.index(length)].append(solution)
        else:
            if length >= min_length:
                grouped_solutions.append([solution])
                dicpos.append(length)
            else:
                grouped_solutions.insert(0, [solution])
                dicpos.insert(0, length)
        min_length = min(length, min_length)

    for length in grouped_solutions:
        grouped_ranked.extend(ag.rank_algs(length, count=20))
        grouped_ranked_rotations.extend(ag.rank_algs_rotation(length, count=20))

    data[setups.iloc[i]["Case"]] = grouped_ranked
    data[f"{setups.iloc[i]["Case"]} rotations"] = grouped_ranked_rotations
    print(i)


In [ ]:
import csv

def write_dict_of_lists_to_csv(dictionary, filename):
    # Get all keys
    keys = dictionary.keys()
    
    # Determine the maximum length of lists in the dictionary
    max_length = max(len(lst) for lst in dictionary.values())
    
    # Prepare data for writing to CSV
    rows = []
    for i in range(max_length):
        row = []
        for key in keys:
            # Append the item if it exists, else append an empty string
            if i < len(dictionary[key]):
                row.append(dictionary[key][i])
            else:
                row.append('')
        rows.append(row)
    
    # Write to CSV
    with open(filename, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        
        # Write the header
        csvwriter.writerow(keys)
        
        # Write the data rows
        csvwriter.writerows(rows)


In [ ]:
write_dict_of_lists_to_csv(data, 'L3T.csv')

In [ ]:
ag = AlgGen()

solution = "BR' L' BL' L U' f' L"
all_solutions = [[],[]]

all_solutions[0].extend(ag.generate_variations(ag.unwiden(solution))[0])
all_solutions[1].extend(ag.generate_variations(ag.unwiden(solution))[1])

scored_solutions = ag.filter_solutions(all_solutions)
scored_solutions = sorted(set(scored_solutions), key=lambda x : x[2])

for alg in scored_solutions:
    print(alg[1].strip(), alg[0], f"({str(alg[3])}, {str(alg[2])})")

In [ ]:
solution = "BR' L' BL' L U' f' L"
all_solutions = [[],[]]

all_solutions[0].extend(generate_variations(unwiden(solution))[0])
all_solutions[1].extend(generate_variations(unwiden(solution))[1])

scored_solutions = filter_solutions(all_solutions)
scored_solutions = sorted(set(scored_solutions), key=lambda x : x[2])

for alg in scored_solutions:
    print(alg[1].strip(), alg[0], f"({str(alg[3])}, {str(alg[2])})")

In [ ]:
alg = "F BR BL BR' BL F u' BR' L' F BL F' BL' L F L F L BL' F L F BL' R' BR BL f R f' L F L R' f u' U' R' BL F u' BR F U F' u F U F U F U F u' F' U F'"
EIF2CIF(unwiden(alg))

In [ ]:
# EIF, CIF
all_solutions2 = [[],[]]
solutions2 = []

alg2 = "B U B U R B' R' B U' L' B' L U' B R B' R' B'"
# alg = "R' F R' B' R F' R B"


solutions2.extend(generate_solutions(setup=alg2, start_depth=15, slack=0, max_depth=15))


for solution in solutions2:
    all_solutions2[0].extend(generate_variations(unwiden(solution))[0])
    all_solutions2[1].extend(generate_variations(unwiden(solution))[1])

In [ ]:
# EIF, CIF
all_solutions = [[],[]]
solutions = []

alg = "B L' B' L U B U L B R B' R' L' U' B'"
# alg = "R' F R' B' R F' R B"


solutions.extend(generate_solutions(setup=alg, start_depth=8, slack=5, max_depth=13))


for solution in solutions:
    all_solutions[0].extend(generate_variations(unwiden(solution))[0])
    all_solutions[1].extend(generate_variations(unwiden(solution))[1])

In [ ]:
scored_solutions = filter_solutions(all_solutions)
scored_solutions = sorted(set(scored_solutions), key=lambda x : x[2])

for alg in scored_solutions:
    print(alg[1].strip(), alg[0], f"({str(alg[3])}, {str(alg[2])})")
    # print(alg[0], alg[1], alg[2], alg[3])

In [ ]:
import subprocess


## Solution Generation

In [ ]:
import subprocess

def generate_solutions_twizzle(setup, count=1000, start_depth=4, max_depth=15, memory=4096):

    AUFs = [("", ""), ("U ", ""), ("U' ", ""), ("", " U"), ("U ", " U"), ("U' ", " U"), ("", " U'"), ("U ", " U'"), ("U' ", " U'")]

    with open("./twizzle/l3t.scr", 'w') as f:
        for auf in AUFs:
            f.write("ScrambleAlg L3T\n")
            f.write(auf[0] + setup + auf[1] + "\n")
            f.write("End\n")

    twsearch_path = "/home/aedan/Desktop/cubing/twsearch/build/bin/twsearch"

    p = subprocess.Popen(f"{twsearch_path} -M {memory} -c {count} --maxdepth {max_depth} --quiet twizzle/FTO.tws twizzle/l3t.scr", shell = True, stdout=subprocess.PIPE)

    pOut = p.communicate()
    p.kill()

    pOut = pOut[0].decode().split("\n")[1:-2]
    pOut = [x.strip() for x in pOut if "No solution found in" not in x]

    return pOut

In [ ]:
pOut = generate_solutions_twizzle("B U B U R B' R' B U' L' B' L U' B R B' R' B'", memory=1000)

In [ ]:
len(pOut)

In [ ]:
pOut

In [ ]:
def EIF2CIF(case):
    CIF = ["U", "D", "F", "B", "R", "BL", "L", "BR"]
    EIF = ["U", "D", "R", "BL", "BR", "L", "F", "B"]
    
    new_case = ""
    case.strip()
    
    for move in case.split(" "):
        suffix = ""
        if "'" in move:
            move = move.removesuffix("'")
            suffix = "'"
        new_case += " " + CIF[EIF.index(move)] + suffix
    return new_case.strip()


def CIF2EIF(case):
    CIF = ["U", "D", "F", "B", "R", "BL", "L", "BR"]
    EIF = ["U", "D", "R", "BL", "BR", "L", "F", "B"]
    
    new_case = ""
    case.strip()
    
    for move in case.split(" "):
        suffix = ""
        if "'" in move:
            move = move.removesuffix("'")
            suffix = "'"
        new_case += " " + EIF[CIF.index(move)] + suffix
    return new_case.strip()


# Rotates arount R face in EIF notation
def x_rotation(case):
    start_orientation = ["U", "D", "F", "B", "R", "BL", "L", "BR"]
    final_orientation = ["B", "F", "BR", "L", "R", "BL", "U", "D"]

    new_case = ""
    case.strip()
    
    for move in case.split(" "):
        suffix = ""
        if "'" in move:
            move = move.removesuffix("'")
            suffix = "'"
        new_case += " " + final_orientation[start_orientation.index(move)] + suffix
    return new_case.strip()


# Rotates arount U face
def y_rotation(case):
    start_orientation = ["U", "D", "F", "B", "R", "BL", "L", "BR"]
    final_orientation = ["U", "D", "BL", "R", "L", "BR", "B", "F"]

    new_case = ""
    case.strip()
    
    for move in case.split(" "):
        suffix = ""
        if "'" in move:
            move = move.removesuffix("'")
            suffix = "'"
        new_case += " " + final_orientation[start_orientation.index(move)] + suffix
    return new_case.strip()

# Rotates around UFR corner
def tr_rotation(case):
    start_orientation = ["U", "D", "F", "B", "R", "BL", "L", "BR"]
    final_orientation = ["BR", "L", "U", "D", "F", "B", "BL", "R"]

    new_case = ""
    case.strip()
    
    for move in case.split(" "):
        suffix = ""
        if "'" in move:
            move = move.removesuffix("'")
            suffix = "'"
        new_case += " " + final_orientation[start_orientation.index(move)] + suffix
    return new_case.strip()

# unwiden algs written with f and u moves
def unwiden(case):
    start_centers = ["U", "u", "R", "BL", "BR", "L", "F", "f"]
    current_centers = ["U", "D", "R", "BL", "BR", "L", "F", "B"]

    new_case = ""
    case.strip()

    for move in case.split(" "):
        suffix = ""
        if "'" in move:
            move = move.removesuffix("'")
            suffix = "'"
        
        if move == "f"  and suffix != "'":
            current_centers[0], current_centers[2], current_centers[5] = current_centers[5], current_centers[0], current_centers[2]
            current_centers[1], current_centers[3], current_centers[4] = current_centers[4], current_centers[1], current_centers[3]
            new_case += " " + current_centers[start_centers.index(move)] + suffix
        elif move == "u" and suffix != "'":
            current_centers[2], current_centers[5], current_centers[7] = current_centers[7], current_centers[2], current_centers[5]
            current_centers[3], current_centers[4], current_centers[6] = current_centers[6], current_centers[3], current_centers[4]
            new_case += " " + current_centers[start_centers.index(move)] + suffix
        elif move == "f" and suffix == "'":
            current_centers[0], current_centers[2], current_centers[5] = current_centers[2], current_centers[5], current_centers[0]
            current_centers[1], current_centers[3], current_centers[4] = current_centers[3], current_centers[4], current_centers[1]
            new_case += " " + current_centers[start_centers.index(move)] + suffix
        elif move == "u" and suffix == "'":
            current_centers[2], current_centers[5], current_centers[7] = current_centers[5], current_centers[7], current_centers[2]
            current_centers[3], current_centers[4], current_centers[6] = current_centers[4], current_centers[6], current_centers[3]
            new_case += " " + current_centers[start_centers.index(move)] + suffix
        else:
            new_case += " " + current_centers[start_centers.index(move)] + suffix
    
    return new_case.strip()

# widen algs to write with f and u moves
def widen(case):
    start_centers = ["U", "D", "R", "BL", "BR", "L", "F", "B"]
    current_centers = ["U", "u", "R", "BL", "BR", "L", "F", "f"]

    new_case = ""
    case.strip()

    for move in case.split(" "):
        suffix = ""
        if "'" in move:
            move = move.removesuffix("'")
            suffix = "'"
        
        if move == "B"  and suffix == "'":
            current_centers[0], current_centers[2], current_centers[5] = current_centers[5], current_centers[0], current_centers[2]
            current_centers[1], current_centers[3], current_centers[4] = current_centers[4], current_centers[1], current_centers[3]
            new_case += " " + current_centers[start_centers.index(move)] + suffix
        elif move == "D" and suffix == "'":
            current_centers[2], current_centers[5], current_centers[7] = current_centers[7], current_centers[2], current_centers[5]
            current_centers[3], current_centers[4], current_centers[6] = current_centers[6], current_centers[3], current_centers[4]
            new_case += " " + current_centers[start_centers.index(move)] + suffix
        elif move == "B" and suffix != "'":
            current_centers[0], current_centers[2], current_centers[5] = current_centers[2], current_centers[5], current_centers[0]
            current_centers[1], current_centers[3], current_centers[4] = current_centers[3], current_centers[4], current_centers[1]
            new_case += " " + current_centers[start_centers.index(move)] + suffix
        elif move == "D" and suffix != "'":
            current_centers[2], current_centers[5], current_centers[7] = current_centers[5], current_centers[7], current_centers[2]
            current_centers[3], current_centers[4], current_centers[6] = current_centers[4], current_centers[6], current_centers[3]
            new_case += " " + current_centers[start_centers.index(move)] + suffix
        else:
            new_case += " " + current_centers[start_centers.index(move)] + suffix
    
    return new_case.strip()

# turns BL moves into Rw moves (EIF)
def rwiden(case):
    start_centers = ["U", "D", "R", "BL", "BR", "L", "F", "B"]
    current_centers = ["U", "D", "R", "Rw", "BR", "L", "F", "B"]

    new_case = ""
    case.strip()

    for move in case.split(" "):
        suffix = ""
        if "'" in move:
            move = move.removesuffix("'")
            suffix = "'"
        
        if move == "BL"  and suffix != "'":
            current_centers[1], current_centers[4], current_centers[6] = current_centers[6], current_centers[1], current_centers[4]
            current_centers[0], current_centers[7], current_centers[5] = current_centers[7], current_centers[5], current_centers[0]
            new_case += " " + current_centers[start_centers.index(move)] + suffix
        elif move == "BL" and suffix == "'":
            current_centers[1], current_centers[4], current_centers[6] = current_centers[4], current_centers[6], current_centers[1]
            current_centers[0], current_centers[7], current_centers[5] = current_centers[5], current_centers[0], current_centers[7]
            new_case += " " + current_centers[start_centers.index(move)] + suffix
        else:
            new_case += " " + current_centers[start_centers.index(move)] + suffix
    
    return new_case.strip()

# turns BL moves into Rw moves (CIF)
def rwiden_CIF(case):
    start_centers = ["U", "D", "R", "BL", "BR", "L", "F", "B"]
    current_centers = ["U", "D", "R", "Rw", "BR", "L", "F", "B"]

    new_case = ""
    case.strip()

    for move in case.split(" "):
        suffix = ""
        if "'" in move:
            move = move.removesuffix("'")
            suffix = "'"
        
        if move == "BL"  and suffix != "'":
            current_centers[1], current_centers[7], current_centers[5] = current_centers[5], current_centers[1], current_centers[7]
            current_centers[0], current_centers[4], current_centers[6] = current_centers[4], current_centers[6], current_centers[0]
            new_case += " " + current_centers[start_centers.index(move)] + suffix
        elif move == "BL" and suffix == "'":
            current_centers[1], current_centers[7], current_centers[5] = current_centers[7], current_centers[5], current_centers[1]
            current_centers[0], current_centers[4], current_centers[6] = current_centers[6], current_centers[0], current_centers[4]
            new_case += " " + current_centers[start_centers.index(move)] + suffix
        else:
            new_case += " " + current_centers[start_centers.index(move)] + suffix
    
    return new_case.strip()

# variations [0] is EIF, [1] is CIF
def generate_variations(case, U_Face="U"):
    rotations = []
    variations = [[], []]
    AUF = ""

    # if case.split()[0] ==  U_Face:
    #     AUF = U_Face
    # elif case.split()[0] == U_Face + "'":
    #     AUF = U_Face + "'"

    case = case.removeprefix(U_Face + "'")
    case = case.removesuffix(U_Face + "'") 
    case = case.removeprefix(U_Face)
    case = case.removesuffix(U_Face)
    
    case = case.strip()

    # white top
    rotations.append((case, AUF + " "))
    rotations.append((y_rotation(case), AUF + " Uo"))
    rotations.append((y_rotation(y_rotation(case)), AUF + " Uo'"))

    # orange top
    rotations.append((x_rotation(case), AUF + " Ro"))
    rotations.append((x_rotation(y_rotation(case)), AUF + " Uo Ro"))
    rotations.append((x_rotation(y_rotation(y_rotation(case))), AUF + " Ro Uo'"))

    # grey top
    rotations.append((x_rotation(x_rotation(case)), AUF + " Ro'"))
    rotations.append((x_rotation(x_rotation(y_rotation(case))), AUF + " Ro' Uo"))
    rotations.append((x_rotation(x_rotation(y_rotation(y_rotation(case)))), AUF + " Ro' Uo'"))

    # green top
    rotations.append((y_rotation(x_rotation(case)), AUF + " Lo'"))
    rotations.append((y_rotation(x_rotation(y_rotation(case))), AUF + " Tr2"))
    rotations.append((y_rotation(x_rotation(y_rotation(y_rotation(case)))), AUF + " Fo"))

    # other orbit
    rotations.append((tr_rotation(case), AUF + " Tr"))
    rotations.append((tr_rotation(y_rotation(case)), AUF + " Tr Uo"))
    rotations.append((tr_rotation(y_rotation(y_rotation(case))), AUF + " Tr Uo'"))

    rotations.append((tr_rotation(x_rotation(case)), AUF + " Tr Ro"))
    rotations.append((tr_rotation(x_rotation(y_rotation(case))), AUF + " Tr Ro Uo"))
    rotations.append((tr_rotation(x_rotation(y_rotation(y_rotation(case)))), AUF + " Tr Ro Uo'"))

    rotations.append((tr_rotation(x_rotation(x_rotation(case))), AUF + " Tr Ro'"))
    rotations.append((tr_rotation(x_rotation(x_rotation(y_rotation(case)))), AUF + " Tr Ro' Uo"))
    rotations.append((tr_rotation(x_rotation(x_rotation(y_rotation(y_rotation(case))))), AUF + " Tr Ro' Uo'"))

    rotations.append((tr_rotation(y_rotation(x_rotation(case))), AUF + " Tr Lo'"))
    rotations.append((tr_rotation(y_rotation(x_rotation(y_rotation(case)))), AUF + " Tr'"))
    rotations.append((tr_rotation(y_rotation(x_rotation(y_rotation(y_rotation(case))))), AUF + " Tr Fo"))

    for orientation in rotations:
        variations[0].append(orientation)
        variations[0].append((rwiden(orientation[0]), orientation[1]))
        variations[1].append((EIF2CIF(orientation[0]), orientation[1]))
        variations[1].append((rwiden_CIF(EIF2CIF(orientation[0])), orientation[1]))


    return variations


def filter_solutions(algs):
    filtered = []

    for alg in algs[0]:
        score = 0
        move_count = 0


        for move in alg[0].split():
            move = move.removesuffix("'")
            move_count += 1

            if move == "B":
                score += 5
            elif move == "F":
                score += 4
            elif move == "BL":
                score += 4
            elif move == "L":
                score += 3
            elif move == "BR":
                score += 2
            elif move == "R":
                score += 1
            elif move == "U":
                score += 1
            elif move == "Rw":
                score += 1
            elif move == "D":
                score += 1.5

        score -= alg.count("R U' R' U") * 2
        score -= alg.count("U' R U R'") * 2
        score -= alg.count("U R' U' R") * 2
        score -= alg.count("R' U R U'") * 2
        score -= alg.count("U R U' R'") * 2
        score -= alg.count("R U R' U'") * 2
        score -= alg.count("R' U' R U") * 2
        score -= alg.count("U' R' U R") * 2


        score -= alg.count("R U' R'") * 1 - alg.count("R U' R' U") * 1 - alg.count("U R U' R'") * 1
        score -= alg.count("U' R U") * 1 - alg.count("U' R U R'") * 1 - alg.count("R' U' R U") * 1
        score -= alg.count("U R' U'") * 1 - alg.count("U R' U' R") * 1 - alg.count("R U R' U'") * 1
        score -= alg.count("R' U R") * 1 - alg.count("R' U R U'") * 1 - alg.count("U' R' U R") * 1
        score -= alg.count("U R U'") * 1 - alg.count("U R U' R'") * 1 - alg.count("R U' R' U") * 1
        score -= alg.count("R U R'") * 1 - alg.count("R U R' U'") * 1 - alg.count("U R' U' R") * 1
        score -= alg.count("R' U' R") * 1 - alg.count("R' U' R U") * 1 - alg.count("U' R U R'") * 1
        score -= alg.count("U' R' U") * 1 - alg.count("U' R' U R") * 1 - alg.count("R' U R U'") * 1

        score -= alg.count("BR BL' BR' BL") * 8
        score -= alg.count("BL' BR BL BR'") * 8
        score -= alg.count("BL BR' BL' BR") * 8
        score -= alg.count("BR' BL BR BL'") * 8
        score -= alg.count("BL BR BL' BR'") * 8
        score -= alg.count("BR BL BR' BL'") * 8
        score -= alg.count("BR' BL' BR BL") * 8
        score -= alg.count("BL' BR' BL BR") * 8

        score -= alg.count("BR BL' BR'") * 4 - alg.count("BR BL' BR' BL") * 4 - alg.count("BL BR BL' BR'") * 4
        score -= alg.count("BL' BR BL") * 4 - alg.count("BL' BR BL BR'") * 4 - alg.count("BR' BL' BR BL") * 4
        score -= alg.count("BL BR' BL'") * 4 - alg.count("BL BR' BL' BR") * 4 - alg.count("BR BL BR' BL'") * 4
        score -= alg.count("BR' BL BR") * 4 - alg.count("BR' BL BR BL'") * 4 - alg.count("BL' BR' BL BR") * 4
        score -= alg.count("BL BR BL'") * 4 - alg.count("BL BR BL' BR'") * 4 - alg.count("BR BL' BR' BL") * 4
        score -= alg.count("BR BL BR'") * 4 - alg.count("BR BL BR' BL'") * 4 - alg.count("BL BR' BL' BR") * 4
        score -= alg.count("BR' BL' BR") * 4 - alg.count("BR' BL' BR BL") * 4 - alg.count("BL' BR BL BR'") * 4
        score -= alg.count("BL' BR' BL") * 4 - alg.count("BL' BR' BL BR") * 4 - alg.count("BR' BL BR BL'") * 4

        score -= alg.count("BR F' BR' F") * 8
        score -= alg.count("F' BR F BR'") * 8
        score -= alg.count("F BR' F' BR") * 8
        score -= alg.count("BR' F BR F'") * 8
        score -= alg.count("F BR F' BR'") * 8
        score -= alg.count("BR F BR' F'") * 8
        score -= alg.count("BR' F' BR F") * 8
        score -= alg.count("F' BR' F BR") * 8

        score -= alg.count("BR F' BR'") * 4 - alg.count("BR F' BR' F") * 4 - alg.count("F BR F' BR'") * 4
        score -= alg.count("F' BR F") * 4 - alg.count("F' BR F BR'") * 4 - alg.count("BR' F' BR F") * 4
        score -= alg.count("F BR' F'") * 4 - alg.count("F BR' F' BR") * 4 - alg.count("BR F BR' F'") * 4
        score -= alg.count("BR' F BR") * 4 - alg.count("BR' F BR F'") * 4 - alg.count("F' BR' F BR") * 4
        score -= alg.count("F BR F'") * 4 - alg.count("F BR F' BR'") * 4 - alg.count("BR F' BR' F") * 4
        score -= alg.count("BR F BR'") * 4 - alg.count("BR F BR' F'") * 4 - alg.count("F BR' F' BR") * 4
        score -= alg.count("BR' F' BR") * 4 - alg.count("BR' F' BR F") * 4 - alg.count("F' BR F BR'") * 4
        score -= alg.count("F' BR' F") * 4 - alg.count("F' BR' F BR") * 4 - alg.count("BR' F BR F'") * 4

        filtered.append((alg[0], "(EIF)", score, move_count))

    for alg in algs[1]:
        score = 0
        move_count = 0

        for move in alg[0].split():
            move_count += 1
            move = move.removesuffix("'")

            if move == "B":
                score += 2
            elif move == "F":
                score += 5
            elif move == "BL":
                score += 5
            elif move == "L":
                score += 2
            elif move == "BR":
                score += 3
            elif move == "R":
                score += 1
            elif move == "U":
                score += 1
            elif move == "Rw":
                score += 1
            elif move == "D":
                score += 3

        score -= alg.count("Rw R'") * 0.5
        score -= alg.count("Rw' R") * 0.5
        score -= alg.count("R Rw'") * 0.5
        score -= alg.count("R' Rw") * 0.5


        score -= alg.count("R B' R' B") * 4
        score -= alg.count("B' R B R'") * 4
        score -= alg.count("B R' B' R") * 4
        score -= alg.count("R' B R B'") * 4
        score -= alg.count("B R B' R'") * 4
        score -= alg.count("R B R' B'") * 4
        score -= alg.count("R' B' R B") * 4
        score -= alg.count("B' R' B R") * 4

        score -= alg.count("R' L R L'") * 4
        score -= alg.count("L R' L' R") * 4
        score -= alg.count("R L' R' L") * 4
        score -= alg.count("L' R L R'") * 4
        score -= alg.count("L' R' L R") * 4
        score -= alg.count("R' L' R L") * 4
        score -= alg.count("L R L' R'") * 4
        score -= alg.count("R L R' L'") * 4


        score -= alg.count("R B' R'") * 2 - alg.count("R B' R' B") * 2 - alg.count("B R B' R'") * 2
        score -= alg.count("B' R B") * 2 - alg.count("B' R B R'") * 2 - alg.count("R' B' R B") * 2
        score -= alg.count("B R' B'") * 2 - alg.count("B R' B' R") * 2 - alg.count("R B R' B'") * 2
        score -= alg.count("R' B R") * 2 - alg.count("R' B R B'") * 2 - alg.count("B' R' B R") * 2
        score -= alg.count("B R B'") * 2 - alg.count("B R B' R'") * 2 - alg.count("R B' R' B") * 2
        score -= alg.count("R B R'") * 2 - alg.count("R B R' B'") * 2 - alg.count("B R' B' R") * 2
        score -= alg.count("R' B' R") * 2 - alg.count("R' B' R B") * 2 - alg.count("B' R B R'") * 2
        score -= alg.count("B' R' B") * 2 - alg.count("B' R' B R") * 2 - alg.count("R' B R B'") * 2

        score -= alg.count("R' L R") * 2 - alg.count("R' L R L'") * 2 - alg.count("L' R' L R") * 2
        score -= alg.count("L R' L'") * 2 - alg.count("L R' L' R") * 2 - alg.count("R L R' L'") * 2
        score -= alg.count("R L' R'") * 2 - alg.count("R L' R' L") * 2 - alg.count("L R L' R'") * 2
        score -= alg.count("L' R L") * 2 - alg.count("L' R L R'") * 2 - alg.count("R' L' R L") * 2
        score -= alg.count("L' R' L") * 2 - alg.count("L' R' L R") * 2 - alg.count("R' L R L'") * 2
        score -= alg.count("R' L' R") * 2 - alg.count("R' L' R L") * 2 - alg.count("L' R L R'") * 2
        score -= alg.count("L R L'") * 2 - alg.count("L R L' R'") * 2 - alg.count("R L' R' L") * 2
        score -= alg.count("R L R'") * 2 - alg.count("R L R' L'") * 2 - alg.count("L R' L' R") * 2

        score -= alg.count("R' D R D'") * 5
        score -= alg.count("D R' D' R") * 4
        score -= alg.count("R D' R' D") * 5
        score -= alg.count("D' R D R'") * 4
        score -= alg.count("D' R' D R") * 5
        score -= alg.count("D R D' R'") * 5


        score -= alg.count("R' D R") * 2 - alg.count("R' D R D'") * 2 - alg.count("D R D' R'") * 2
        score -= alg.count("D R' D'") * 2 - alg.count("D R' D' R") * 2
        score -= alg.count("R D' R'") * 2 - alg.count("R D' R' D") * 2 - alg.count("D' R' D R") * 2
        score -= alg.count("D' R D") * 2 - alg.count("D' R D R'") * 2
        score -= alg.count("D' R' D") * 2 - alg.count("D' R' D R") * 2 - alg.count("R D' R' D") * 2
        score -= alg.count("D R D'") * 2 - alg.count("D R D' R'") * 2 - alg.count("R' D R D'") * 2

        filtered.append((alg[0], "(CIF)", score, move_count))


    return filtered

In [ ]:
EIF2CIF(unwiden("BL' u F BR u' F BL BR L' u' BR u' BL u R' u' f BL BR' f' BL' R' U R BL BR L' U' R U L' f' BL' F' U BR u BL BR F L' BL L U BR"))